# Project: ICEYE test on flood - Draft Plan

## 1. Data Acquisition and Preprocessing

### Chosen Region: Kuantan, Pahang, - Malaysia

**Justification:**
- **High Historical Flood Frequency:** Frequently affected by severe floods, particularly during the northeast monsoon season.
- **Moderate Disaster Insurance Gap:** Many residents, of both poor and middle-class background - giving reason to approach this region to help the poor and vulnerable while  navigating the constraints of the capitalist system.
- **High Commercial Potential for Flood-Extent Products:** Valuable for insurance companies, government agencies, and disaster relief organizations to improve resource allocation and emergency response strategies.
- **High Potential for Flood-Extent Products:** Flood-extent products are a key component of flood risk management, and are used by insurance companies, government agencies, and disaster relief organizations to improve resource allocation and emergency response strategies.

**Kuantan, Pahang:**
Kuantan district, along with Temerloh and Pekan, is highly vulnerable to floods in Pahang state. [Source](https://www.oecd-ilibrary.org/sites/2621c072-en/index.html?itemId=%2Fcontent%2Fcomponent%2F2621c072-en)
Kuantan faces significant flood risks due to its natural setting near the Kuantan River basin, changing climate, and development pressures. [Source](https://www.oecd-ilibrary.org/sites/2621c072-en/index.html?itemId=%2Fcontent%2Fcomponent%2F2621c072-en)



### B. Data Sources
- **DTM Data:**
  - **Source:** Copernicus Open Access Hub (Sentinel).
  - **Resolution:** High-resolution DTMs, processed to 10 meters.
  - **Availability:** Freely accessible, comprehensive data.
    - [Source](https://scihub.copernicus.eu/)
- **SAR Data:**
  - **Source:** Sentinel-1 via Copernicus Open Access Hub.
  - **Frequency:** High-resolution C-band SAR data with a revisit time of 6 to 12 days.
    - [Source](https://scihub.copernicus.eu/)
- **Meteorological Data:**
  - **Source:** METMalaysia and data.gov.my for weather patterns and conditions.
  - **Availability:** Accessible through APIs and relevant meteorological services.
    - [Source](https://met.gov.my/)
    - [Source](https://data.gov.my/)

### C. Preprocessing Steps
1. **Download Data:**
   - Obtain 30-meter resolution DTM data 
   - Acquire Sentinel-1 SAR data and meteorological data and resample to 30 meters.
2. **Resampling and Normalization:**
   - Use down sampling techniques to resample SAR data to 30 meters. (I'm not yet completely sure what effect this will have on the result - going with SAR resolution as the "lowest common denominator" might prove to be the optimal case
   - Normalize SAR and meteorological data for consistency.
3. **Handling Missing Data:**
   - Apply interpolation or other methods to fill gaps in the datasets. If certain modalities are missing mith a frequency of 10-50% we can apply modality-wise drop out - making the model more robust to regions with limited multi-modal data,
4. **Data Conversion:**
   - Convert resampled and normalized data into the desired format (e.g., GeoTIFF (for later vis) / hdf5 or .pt [for data loading)).
  - Quantise the data if necessary, depending on fine-tune/training computational req. and speed bfloat16 likely to be ideal post norm.

## 2. Feature Engineering and Modeling

### A. Feature Engineering
- **Relationship Analysis:**
  - Examine the connections between historical flood locations, Sentinel-1 revisit times, flood characteristics (e.g., size, duration), terrain data, and weather patterns.
- **Feature Development:**
  - Develop features predicting the probability of capturing a flood event within 24 hours.
- **Feature Representation:**
  - Given that our chosen model finds relationships within the time slice of the data and between them, we can represent each datapoint as a "time node" with DTM/SAR/Met features rolled out into a flat matrix. Such that the input to the model is a matrix of size (N<sub>k</sub>,T<sub>hr</sub>) where N<sub>1</sub> could be DTM/SAR/SAR<sub>sigma</sub>/SAR<sub>NDWI</sub> pixels or a certain time-series signal for the given area.

### B. Modeling Approach
- **Model Selection:**
  - Utilize the SoTA PatchTSMixer model, configured for multivariate time series forecasting. [Source](https://huggingface.co/ibm-granite/granite-timeseries-patchtsmixer)
    - ![PatchTSMixer](resources/img.png)
        - 
    - Reason for selection: One can split time series forcasting into univariate (i.e. single signal) and multivariate (i.e. multiple signals) problems. The PatchTSMixer model is a multivariate model that can handle both. Not only that, it exceeds the previous state of the art by 8 - 60% in Mean Squared Error (MSE) which is quite massive.
    - The design of the architecture is such that it does not care what physical phenomena or spatial distance your signal is away from the target. It is a "patch" model that can be applied to a combination of many different modalitites (unrolled imagerys akeen to DETR [one of the first transformer architectures to use transformers] for SAR, DTM for DTM, etc.) and can be trained on a variety of datasets.
    - Ideally the architecture should be slightly modified to be more akeen to auto-regressive models. Such that the input to the model can be coninous signals of various modalities at every time step, and the output is a single continuous signal. At present, the model is trained on a 1-hour time-step *with a 21-day context window* (i.e. 21 hours of data before now can be fed into it). We also had to adopt to 7 features to represent the 7 modalities in the pre-trained model (I was hoping to get this running but only had less than a day and ran out of time) - in reality we can incorporate far more features at every time step, depending on the region size and models size applicability.
    - THe additional benefit of using transformers is the insane acceleration of fine-tuning and transfer learning techniques; as an example we can borrow quantised  2,4,5,6,8 bit CUDA kernels from llama.cpp and apply them to the model.
    - With techniques such at DPO and QLoRA fine-tuning, the model can constantly evolve without encountering catastrophic forgetting.

- **Training Data:**
  - Generate sequences of tokens representing 1-hour (or less) integrals of data from all modalities.
  - Apply dropout in the input token layer to enhance robustness.
    - 
- **Evaluation:**
  - Assess model performance using metrics like AUC-ROC 2x2 conf mat, and F1-score (which is just binary accuracy in this case).
  - Given time to modify the architecture into an autoregressive one, we can leverage each floor even instance (be it 3 hours or 24 hours) to train the model on a 21-day window of data to predict the next token of probability of a flood, and let it run for N time steps to try to make projections into the future. This would be akin to a "forecast" of the flood probability for the next 21 days that can take into account any modality no matter how physically or spatially distant it is from the others.
  - Au auto-regressive architecture could also let you determine the accuracy of going into a new area. As we can see if feeding in the 21-day window of data from the new area, can predict the probability of flooding based on historical data.
  - The strength of each signal inut and its predictive ability for the model can also we further evaluated by looking at the importance of each feature in the model, by levraging techniques like PCA.
  - Model Calibration: By calibrating the model probability at each time step (after the feature-wise softmax) we can get a more accurate probability of flooding, rather than just a number between 0 and 1 which has no resemblance to the actual probability of flooding (always important to explain this factoid to clients)

Get the geopolygon of the area in question, calculate total area in sqr km, output to KML to sanity check with google earth

In [1]:
import geojson
import shapely
import simplekml
import pyproj
import geopandas as gpd
from shapely.geometry import Polygon, shape
from shapely.ops import transform
from functools import partial
import math
from shapely.ops import transform

def calculate_area_using_utm(input_geometry):
    """
    Calculate the area of the input geometry in square meters using UTM projection.
    transformation from WGS84 to UTM.
    """
    from_epsg = 4326
    to_epsg = get_utm_epsg(shapely.Point(input_geometry.centroid))

    transformation = pyproj.Transformer.from_crs(
        crs_from=pyproj.CRS.from_epsg(from_epsg),
        crs_to=pyproj.CRS.from_epsg(to_epsg),
        always_xy=True)

    project = lambda x, y: transformation.transform(x, y)
    transformed_geom = transform(project, input_geometry)

    return transformed_geom.area

def get_utm_epsg(point):
    """
    Get the EPSG code for the UTM zone corresponding to a given WGS84 point.

    Args:
        point (shapely.Point): The point for which to determine the UTM zone.

    Returns:
        int: The EPSG code for the UTM zone.
    """
    # Get the latitude and longitude of the point
    lon = float(point.x)
    lat = float(point.y)
    zone_number = int(math.floor((lon + 180) / 6) + 1)
    
    # More accurate EPSG code for the Kuantan region (took a while to remember this xD)
    if (0 <= lat < 7) or (14 <= lat < 21):
        epsg_code = '327' + str(zone_number)
    else:
        epsg_code = '326' + str(zone_number)
        
    return epsg_code




# Specify the region name and coordinates
region_name = "Kuantan"

# Note polygon chosen using map-box - if proven successful, we can extend the polygon to include more area and determine generalization
poc_extent = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "coordinates": [
          [
            [103.30258965042253, 3.8536955341197654],
            [103.30258965042253, 3.761091245587309],
            [103.39347050856702, 3.761091245587309],
            [103.39347050856702, 3.8536955341197654],
            [103.30258965042253, 3.8536955341197654],
            [103.30258965042253, 3.8536955341197654]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}





# Create a Feature object
kuantan = shapely.geometry.Polygon.from_bounds(poc_extent['features'][0]['geometry']['coordinates'][0][0][0], poc_extent['features'][0]['geometry']['coordinates'][0][0][1], poc_extent['features'][0]['geometry']['coordinates'][0][2][0], poc_extent['features'][0]['geometry']['coordinates'][0][2][1])

# Sanity check rea so our NN is not massive
kuantan_area = calculate_area_using_utm(kuantan)
print(kuantan_area) # 114_860 30x30m tiles

103373524.18945646


2, ## Modalities and Their Importance for Flood Prediction

**Sentinel-1 Satellite Data (Radar Imagery):**

* **Importance:** Radar imagery from the Sentinel-1 satellite is crucial for detecting changes in land surface and water bodies. It provides information about soil moisture, surface roughness, and standing water, which are essential indicators of flood conditions.
* **Processing:** The raw amplitude data (VV and VH polarizations) are normalized to ensure consistent scaling and comparability across images and time points.

**Copernicus DEM (Digital Elevation Model):**

* **Importance:** Elevation and slope data from the Copernicus DEM offer critical insights into the topography of the area. This information is essential for understanding water flow patterns and identifying areas prone to flooding.
* **Processing:**  Elevation and slope data are normalized for consistency with other modalities and to aid the PatchTSMixer model's learning process.

**METMalaysia Weather Data:**

* **Importance:** Meteorological factors, particularly precipitation and temperature, play a significant role in flood events. High rainfall and elevated temperatures can lead to increased runoff and saturated soil conditions, increasing flood risk.
* **Processing:** Precipitation and temperature data are normalized to align with model input requirements and ensure balanced feature importance.

**Why This Specific Preprocessing?**

The normalization step serves several crucial purposes:

* **Comparability:** Allows comparison of data from diverse sources and time points on a common scale.
* **Model Performance:** Enhances model convergence and stability, leading to faster training and improved performance.
* **Feature Importance:** Prevents features with larger scales from dominating, ensuring all features contribute meaningfully to the prediction.

**Additional Considerations for PatchTSMixer:**

* **Channel Mapping:** Careful mapping of collected data to the seven channels expected by the PatchTSMixer model is required.
* **Missing Data Handling:** The model's robustness to missing data is beneficial, but strategies for imputation or interpolation may be necessary if significant gaps exist in the collected data.


In [2]:
from data.data_collector import FloodDataCollector
from datetime import datetime, timedelta

start_date = datetime(2010, 1, 1)
end_date = datetime(2024, 1, 31)
resolution = timedelta(hours=1)
collector = FloodDataCollector(kuantan, start_date, end_date, resolution,output_path="train_val.pt")
collector.collect_data()

SyntaxError: invalid syntax (1359084572.py, line 1)

3. Determining the best time chunks for the data (for future development):


Here are the key points regarding the frequency of SAR, DTM, and MET data in Malaysia:

Sentinel-1 SAR:
- The Sentinel-1 constellation has a repeat frequency of 6 days with two satellites, and 12 days with one satellite over Europe.
- The Sentinel-1 C-SAR instrument has a centre frequency of 5.405 GHz.

MET Malaysia weather data:
- MET Malaysia provides daily updates for 7-day general forecast data.
- Weather warning data, including earthquake data, is updated when required by MET Malaysia.
- MET Malaysia has a network of automatic weather stations that provide real-time data, which is then processed at the headquarters to generate forecasts and warnings.
- Historical rainfall data from meteorological stations operated by MET Malaysia is used in various research studies.

Considering the Sentinel-1 SAR has a repeat frequency of 6-12 days, and MET Malaysia provides daily weather updates, I would use a context length that covers at least 12 days of historical data. This would ensure the model can capture the temporal patterns and variability in both the SAR imagery and corresponding weather conditions.

For example, if you have hourly weather data, a 12-day context length would correspond to 288 hours (12 days * 24 hours/day). If you have daily weather data, the context length would be 12 time steps.

The specific context length can be fine-tuned based on the granularity of your weather data and computational constraints. But in general, a 12-day or longer context should allow the model to learn meaningful relationships between the SAR observations and meteorological conditions for flood forecasting in Malaysia.


# Flood Data Collection and Processing

This code is designed to collect and process flood-related data from various sources and create balanced train and validation datasets for flood prediction.

The `FloodDataCollector` class takes a GeoJSON file or dictionary, start and end dates, a time resolution, and an output directory as input. It then performs the following steps:

1. Loads the GeoJSON data and extracts the bounding box coordinates.
2. Retrieves Sentinel-1 satellite data for the specified bounding box and time range.
3. Retrieves Copernicus DEM (Digital Elevation Model) data for the specified bounding box.
4. Retrieves meteorological data from the Met Malaysia API for the specified location and time range.
5. Retrieves flood data from the Open-Meteo API for the specified location and time range.
6. Combines the collected data into a single dataset.
7. Balances the dataset by oversampling the minority class (flood instances) to ensure an equal number of flood and no-flood instances.
8. Splits the balanced dataset into train and validation sets with an 80/20 split.
9. Saves the train and validation datasets as `train_dataset.pt` and `val_dataset.pt` files in the specified output directory.

The code utilizes caching mechanisms to avoid redundant API calls and improve performance. It also applies normalization to the collected data to ensure consistent scaling.

The resulting train and validation datasets can be used for training and evaluating flood prediction models.


In [6]:
import torch
import os
# Assuming the required inputs are loaded in previous cells
geojson_path = "path/to/your/geojson/file.geojson"
start_date = datetime(2010, 1, 1)
end_date = datetime(2024, 1, 31)
resolution = timedelta(hours=1)
output_dir = "output"

# Create an instance of the FloodDataCollector
collector = FloodDataCollector(geojson_path, start_date, end_date, resolution, output_dir)

# Collect and process the data
collector.collect_data()

# Load the generated train and validation datasets
train_dataset = torch.load(os.path.join(output_dir, "train_dataset.pt"))
val_dataset = torch.load(os.path.join(output_dir, "val_dataset.pt"))

TypeError: 'module' object is not callable

In the training cell, we will initialize the `ModelTrainer` class with the provided `FloodForecastingDataLoader` instance. This class is responsible for loading the data and preparing it for training and validation. The `ModelTrainer` class also includes methods for loading a pre-trained model, training the model with the specified parameters (number of epochs, batch size, and learning rate), and saving the trained model.

The training process includes the following steps:
- Preparing the training and validation datasets.
- Loading a pre-trained `PatchTSMixerForPrediction` model.
- Setting up the training arguments, including the number of epochs, batch size, learning rate, and logging directory.
- Initializing the `Trainer` with the model, training arguments, and datasets.
- Executing the training loop using the `trainer.train()` method.

After training, the model is evaluated on the validation dataset to calculate the F1-score, and the results are logged. The trained model is then saved to the specified directory.

In [ ]:
from data.dataloader import FloodForecastingDataLoader
from model.modeltrainer import ModelTrainer
# Initialize the data loader with the required parameters
train_data_loader = FloodForecastingDataLoader(train_dataset, forecast_horizon=96) # last 48 hrs iignored for training 

# Create an instance of the ModelTrainer class
trainer_instance = ModelTrainer(train_data_loader)

# Train the model with the specified parameters
trainer_instance.train()




In the validation cell, we will utilize the `validate_and_evaluate` method of the `ModelTrainer` class to assess the performance of the trained model on the validation dataset. This method performs the following steps:
- It loads the validation dataset into a `DataLoader` with the appropriate batch size.
- It uses the `Trainer` to predict the outcomes on the validation dataset.
- It post-processes the predictions to convert them into binary flood predictions.
- It calculates the F1-score, which is a harmonic mean of precision and recall, and is a measure of the model's accuracy on the binary classification task.
- It generates and saves the ROC curve and confusion matrix plots to visualize the model's performance.

The outputs of this process include:
- F1-score: A single value indicating the model's accuracy.
- ROC curve: A plot showing the relationship between the true positive rate and false positive rate at various threshold settings.
- Confusion matrix: A heatmap showing the true positives (TP), false positives (FP), true negatives (TN), and false negatives (FN).


In [4]:
from model.modeltrainer import ModelTrainer


# Initialize the data loader with the required parameters
validate = ModelTrainer(FloodForecastingDataLoader(val_dataset, forecast_horizon=96))
# Assuming the ModelTrainer instance 'trainer_instance' has already been created and trained in previous cells

# Prepare the validation dataset
_, val_dataset = trainer_instance.prepare_data()

# Load the trained model (this assumes the model has been trained and saved in the previous steps)
model = trainer_instance.load_pretrained_model()

# Validate and evaluate the model, which will also generate the ROC curve and confusion matrix plots
f1_score = trainer_instance.validate_and_evaluate(val_dataset, model)

# Log the results
print(f"Validation F1 Score: {f1_score}")

NameError: name 'trainer_instance' is not defined

# Visual Training Dashboard

Here's how the script works:

The script creates a Dash app with a layout that includes graphs for validation loss, ROC curve, and confusion matrix.
The ModelTrainer class is modified to update the global variables val_loss_data, roc_curve_data, and confusion_matrix_data during the training process.
The run_dashboard function is defined to set up the data loader, create an instance of ModelTrainer, and start the model training in a separate thread.
The Dash app is run using app.run_server(), which starts the dashboard and allows it to update in real-time.
The callbacks for updating the graphs are defined using the @app.callback decorator. These callbacks are triggered at regular intervals (every 5 seconds in this example) to update the graphs with the latest data.

To run the script from a Jupyter notebook, you can use the %run magic command followed by the path to the script file. For example:
This will execute the script and start the Dash app, which you can then access through the provided URL in the notebook output.
Note: Make sure to set the appropriate values for geojson_path, start_date, end_date, forecast_horizon, epochs, batch_size, and learning_rate before running the script.

In [ ]:
%run path/to/dashboard_script.py